In [1]:
#     bw-migrations             0.1                      pypi_0    pypi
#     bw2analyzer               0.10                     pypi_0    pypi
#     bw2calc                   1.8.0                    pypi_0    pypi
#     bw2data                   3.6.2                    pypi_0    pypi
#     bw2io                     0.8.3.1                  pypi_0    pypi
#     bw2parameters             0.6.6                    pypi_0    pypi

import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw2analyzer as ba

import numpy as np
import json, os
from pathlib import Path
import sys



from importlib import reload

#Local files
from consumption_model_ch.import_databases import *

from consumption_model_ch.consumption_fus import (add_consumption_activities,
                                                  add_consumption_hh,
                                                  # add_consumption_average_hh,
                                                  add_household_cluster_consumption,
                                                  add_consumption_sectors,
                                                  add_consumption_categories
                                                 )


In [2]:
bd.projects


Brightway2 projects manager with 8 objects:
	AB_test_exploring_EI39
	CCL dev
	default
	eco-ex
	eco-ex-March-2023
	ecoinvent 3.3 cutoff
	validation
	validation_ei39
Use `projects.report()` to get a report on all projects.

In [3]:
# Some settings
delete_consumption_db = True  #  set True to rebuild the  consumption database
add_functional_units = True
habe_year = '151617'

# Define paths to databases
path_base = Path('/Users/ajakobs/Documents/data/')
directory_habe = path_base / 'HABE/131_EBM 2015-2017/HABE151617_191111UOe/'
fp_ecoinvent_38 = path_base / 'ecoinvent/ecoinvent 3.8_cutoff_ecoSpold02/datasets/'
fp_ecoinvent_33 = path_base / 'ecoinvent/ecoinvent 3.3_cutoff_ecoSpold02/datasets/'
fp_exiobase = path_base / 'EXIOBASE/EX_3_8_1/IOT_2015_pxp/'


In [4]:
project = "CCL dev"
bd.projects.set_current(project)

co_name = "swiss consumption 1.0"  # CONSUMPTION_DB_NAME


# Import all ecoinvent
bi.bw2setup()
ei38_name = "ecoinvent 3.8"
import_ecoinvent(fp_ecoinvent_38, ei38_name)
# Import Exiobase
ex38_name = "exiobase 3.8.1 monetary"
import_exiobase_3(fp_exiobase, ex38_name)


Biosphere database already present!!! No setup is needed
ecoinvent 3.8 database already present!!! No import is needed


In [5]:
%%time
# Import consumptionn db
exclude_databases = [
        'heia',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - {}'.format(ei38_name),
        # 'exiobase 3.8.1 monetary',
    ]
if delete_consumption_db and co_name in bd.databases:
    del bd.databases[co_name]
import_consumption_db(directory_habe, habe_year, co_name, fp_ecoinvent_33, exclude_databases, fp_exiobase)
co = bd.Database(co_name)



Biosphere database already present!!! No setup is needed
ecoinvent 3.3 cutoff database already present!!! No import is needed
--> Consumption_db.xlsx already exists, reading it
Created consumption_db.xlsx file in BW format in 1.13 seconds
Extracted 1 worksheets in 0.83 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applied 0 strategies in 0.00 seconds
207 datasets
6676 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/28/2023 20:55:26
  Finished: 03/28/2023 20:55:27
  Total time elapsed: 00:00:00
  CPU %: 97.20
  Memory %: 2.56
Created database: swiss consumption 1.0
CPU times: user 40.6 s, sys: 7.39 s, total: 47.9 s
Wall time: 51.1 s


In [6]:
bd.databases

Databases dictionary with 5 object(s):
	biosphere3
	ecoinvent 3.8
	exiobase 3.8.1 monetary
	exiobase 3.8.1 monetary biosphere
	swiss consumption 1.0

## Add functional units
- Total and average household consumption
- Add different household cluster
- add different sectors?

### Total and average household demand

In [7]:
# Set option disaggregated to use the preprocessed data! Only option needed for now. 
# Perhaps change this later in the code as I think it is confusing and does not really make sense atm. 
option = 'disaggregated'
file_path = '/Users/ajakobs/Documents/CCL/habe_data_prepared/habe20152017_hh_prepared_imputed.csv' 

In [8]:
%%time
# First add consumption demand processes for all households
add_consumption_hh(co_name, habe_year, directory_habe,  file_path_prepared_habe_data=file_path, option=option)

CPU times: user 2.68 s, sys: 916 ms, total: 3.59 s
Wall time: 3.95 s


In [9]:
demand = [act for act in co if f'consumption {option}' in act['name']]
assert len(demand)==1
demand = demand[0]
demand

'ch hh average consumption disaggregated, years 151617' (1 month of consumption, CH, None)

In [10]:
demand.as_dict()

{'database': 'swiss consumption 1.0',
 'code': 'ch hh average consumption disaggregated, years 151617',
 'location': 'CH',
 'name': 'ch hh average consumption disaggregated, years 151617',
 'unit': '1 month of consumption',
 'agg_option': 'disaggregated',
 'n_households': 9955,
 'year_habe': '151617',
 'id': 35113}

In [11]:
len([act for act in co])

208

### Different household clusters

In [12]:
%%time
# Add different household clusters based on two demographics
fp_clustering = '/Users/ajakobs/Documents/CCL/scripts/notebooks/write_files/ccl_dev/households_clustering.csv'
add_household_cluster_consumption(co_name, habe_year, fp_habe_clustering=fp_clustering)



Creating household cluster functional units
CPU times: user 54.4 s, sys: 26 s, total: 1min 20s
Wall time: 1min 32s


In [13]:
%%time
# Add clusters based on only 1 demographi at a time (income, hh type, or region)
fp_clustering_simple = '/Users/ajakobs/Documents/CCL/scripts/notebooks/write_files/ccl_dev/households_clustering_simple.csv'
add_household_cluster_consumption(co_name, habe_year, fp_habe_clustering=fp_clustering_simple)


Creating household cluster functional units
CPU times: user 14.4 s, sys: 6.18 s, total: 20.5 s
Wall time: 23.4 s


In [14]:
# and reset min max values for all other exchanges, apparently some cause trouble, fixed by Chris with this code:
for ds in bd.Database("swiss consumption 1.0"):
    for exc in ds.technosphere():
        if exc.get('uncertainty type') == 0 or exc.get('uncertainty_type') == 0:
            print(exc.get('name'), exc['minimum'], exc['maximum'])
            # exc['minimum'] = exc['maximum'] = np.NaN
            # exc.save()

In [15]:
len([act for act in co])

314

## backup project

In [18]:
%%time
# back  up
bi.backup_project_directory(project=project)

Creating project backup archive - this could take a few minutes...
CPU times: user 1min, sys: 1.97 s, total: 1min 2s
Wall time: 1min 5s


In [19]:
# stored here: "/Users/ajakobs/brightway2-project-CCL dev-backup.10-March-2023-05-27PM.tar.gz"

In [20]:
%%time
bi.export.write_lci_csv(co_name)

CPU times: user 38.2 s, sys: 650 ms, total: 38.8 s
Wall time: 39 s


'/Users/ajakobs/Library/Application Support/Brightway3/CCL-dev.f4b5e8d8/output/lci-swiss-consumption-10.csv'